In [14]:
from agents.structured_chat import StructuredChatAgent
from tools.base import Tool
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()

def search_func(args):
    input = args.get('input')
    return search.run(input)

search_tool = Tool(**{
    "name": "Current Search",
    "func": search_func,
    "description": "useful for when you need to answer questions about current events or the current state of the world",
    "args": {"tool_input": {"type": "string"}}
})


In [16]:
s = 'AB'
# to lowercase
s.lower()


'ab'

In [17]:
def company_query_func(args):
    name = args.get('tool_input').lower()
    members = {
        "james": "James is a member of esri company",
        "jim": "Jim is a member of google company",
        "jimmy": "Jimmy is a member of openai company",
        "jimothy": "Jimothy is a member facebook company",
    }
    return f"{members[name]}"


company_query_tool = Tool(**{
    "name": "CompanyQuery",
    "func": company_query_func,
    "description": "useful for when you want to know someone's company",
    "args": {"tool_input": {"type": "string"}}
})

def company_info_func(args):
    company = args.get('tool_input').lower()
    companys = {
        "esri": "esri is a geo related company",
        "google": "google is a search engine related company",
    }
    return f"{companys[company]}"


company_info_tool = Tool(**{
    "name": "GetCompanyInfo",
    "func": company_info_func,
    "description": "useful for when you want to know a company's info",
    "args": {"tool_input": {"type": "string"}}
})

In [18]:
input = 'James belongs to which company? Introduce the company\'s information'
tools = [company_query_tool, company_info_tool]
agent = StructuredChatAgent(tools, verbose=True)

In [19]:
agent.run(input=input)

> Start: 
James belongs to which company? Introduce the company's information

Thought:I need to use the CompanyQuery tool to find out which company James belongs to. Then I can use the GetCompanyInfo tool to introduce the company's information. Let me do that now.

Action:
```
{
  "action": "CompanyQuery",
  "action_input": {
    "type": "string",
    "tool_input": "James"
  }
}
```
Observation: James is a member of esri company

Thought:James belongs to the Esri company. Now I can use the GetCompanyInfo tool to introduce the company's information.

Action:
```
{
  "action": "GetCompanyInfo",
  "action_input": {
    "type": "string",
    "tool_input": "Esri"
  }
}
```
Observation: esri is a geo related company

Thought:Esri is a geo-related company. They specialize in geographic information system (GIS) software and technology. They provide mapping and spatial analytics solutions to help organizations make informed decisions. Is there anything specific you would like to know about Esr

'Esri is a geo-related company. They specialize in geographic information system (GIS) software and technology. They provide mapping and spatial analytics solutions to help organizations make informed decisions. Is there anything specific you would like to know about Esri?'